# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from Weather_Vacation_api_keys import weather_api_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv(r"C:\Users\asg_a_1p8y6mm\OneDrive\Desktop\WIOA Training\DataAnalytics\Module 6\Module 6; Class Challenge\python-api-challenge\WeatherPy_&_VacationPy\output_data\cities.csv")

# Display sample data
print('Display sample of saved city_data_df dataframe')
display(city_data_df.head())


Display sample of saved city_data_df dataframe


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sao jose da coroa grande,-8.8978,-35.1478,24.10,80,34,4.42,BR,1731303325
1,1,saipan,15.1355,145.7010,31.39,81,40,5.14,MP,1731303326
2,2,yellowknife,62.4560,-114.3525,-13.71,96,75,4.12,CA,1731303067
3,3,west island,-12.1568,96.8225,28.99,74,40,7.72,CC,1731303328
4,4,puerto natales,-51.7236,-72.4875,5.36,88,96,3.10,CL,1731303330


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot for all cities

# Convert City column to a categorical type for distinct colors
city_data_df["City"] = city_data_df["City"].astype("category")

# Mapping for all cities
map_plot_all = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity",
    scale=1.0,
    color="City",  # Color by city (categorical)
    cmap="Category20", # Use a categorical colormap (adjust number as needed for more categories)
    hover_cols=["City", "Country", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"],
    title="World Cities Map (All)"
)

# Display the map
map_plot_all

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Max Temp,Cloudiness,Wind Speed)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df[
    (city_data_df["Max Temp"] < 27) & 
    (city_data_df["Max Temp"] > 21) &
    (city_data_df["Wind Speed"] < 4.5) &
    (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values (though the prior filtering should handle this)
ideal_city_df = ideal_city_df.dropna()


# Display sample data
print('Display of ideal_city_df dataframe')
display(ideal_city_df)

# You can now use ideal_city_df for your mapping, which will be much faster
# due to the reduced number of points:

# Convert City column to a categorical type for distinct colors
ideal_city_df["City"] = ideal_city_df["City"].astype("category")

# Mapping for filtered cities
map_plot_filtered = ideal_city_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity",
    scale=1.0,
    color="City",  # Color by city
    cmap="Category20", # Categorical colormap
    hover_cols=["City", "Country", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"],
    title="World Cities Map (Filtered)"
)

# Display the map
map_plot_filtered

Display of ideal_city_df dataframe


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
98,98,dakhla,23.6848,-15.9580,21.91,45,0,4.08,EH,1731303444
180,180,goundam,16.4145,-3.6708,21.34,19,0,3.71,ML,1731303550
338,338,leland,34.2563,-78.0447,22.01,81,0,2.06,US,1731303750
426,426,dongyang,29.2678,120.2253,22.18,58,0,1.91,CN,1731303859
514,514,chinde,26.8212,87.7095,26.08,49,0,2.44,NP,1731303968
519,519,san antonio,29.4241,-98.4936,21.45,66,0,2.06,US,1731303975
543,543,nanakuli,21.3906,-158.1547,25.97,68,0,2.06,US,1731304005


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Max Temp,Cloudiness,Wind Speed)

### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df 
# to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
print('Sample of the hotel_df dataframe')
display(hotel_df.head())

Sample of the hotel_df dataframe


,City,Country,Lat,Lng,Humidity,Hotel Name
98,dakhla,EH,23.6848,-15.9580,45,
180,goundam,ML,16.4145,-3.6708,19,
338,leland,US,34.2563,-78.0447,81,
426,dongyang,CN,29.2678,120.2253,58,
514,chinde,NP,26.8212,87.7095,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Import API key
from Weather_Vacation_api_keys import geoapify_key

# Set parameters to search for a hotel
radius = 10000  # 10,000 meters = 10 kilometers
params = {
    "categories": "accommodation.hotel",  # Search for hotels
    "limit": 1,                         # Get only the first result
    "apiKey": geoapify_key            # Your Geoapify API key (replace with your actual key)
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}" # Filter results within the radius
    params["bias"] = f"proximity:{lng},{lat}" # Bias results towards the specified coordinates

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    try:
      # Make and API request using the params dictionary
      name_address = requests.get(base_url, params=params)
      name_address.raise_for_status() # Raise an exception for bad status codes (4xx or 5xx)

      # Convert the API response to JSON format
      name_address = name_address.json()

      # Grab the first hotel from the results and store the name in the hotel_df DataFrame
      hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]

    except (KeyError, IndexError, requests.exceptions.RequestException) as e:  # Handle exceptions
        # If no hotel is found or there's an error, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        print(f"Error finding hotel for {row['City']}: {e}")  # Print the error message


    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
print('Display the hotel_df dataframe which now includes hotel data')
display(hotel_df)

Starting hotel search
dakhla - nearest hotel: Hotel Ayouz
Error finding hotel for goundam: list index out of range
goundam - nearest hotel: No hotel found
leland - nearest hotel: Holiday Inn Express Leland - Wilmington Area
Error finding hotel for dongyang: list index out of range
dongyang - nearest hotel: No hotel found
Error finding hotel for chinde: list index out of range
chinde - nearest hotel: No hotel found
san antonio - nearest hotel: Riverwalk Plaza Hotel and Suites
nanakuli - nearest hotel: Aulani, A Disney Resort & Spa
Display the hotel_df dataframe which now includes hotel data


,City,Country,Lat,Lng,Humidity,Hotel Name
98,dakhla,EH,23.6848,-15.9580,45,Hotel Ayouz
180,goundam,ML,16.4145,-3.6708,19,No hotel found
338,leland,US,34.2563,-78.0447,81,Holiday Inn Express Leland - Wilmington Area
426,dongyang,CN,29.2678,120.2253,58,No hotel found
514,chinde,NP,26.8212,87.7095,49,No hotel found
519,san antonio,US,29.4241,-98.4936,66,Riverwalk Plaza Hotel and Suites
543,nanakuli,US,21.3906,-158.1547,68,"Aulani, A Disney Resort & Spa"


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# Configure the map plot
# Mapping for filtered cities with hotel information in hover
map_plot_filtered_hotels = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity",
    scale=1.0,  # Adjust as needed
    color="City",       # Color by city
    cmap="Category20",  # Use a categorical colormap
    hover_cols=["City", "Country", "Hotel Name", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"], # Include "Hotel Name"
    title="Ideal Cities Map with Hotels"
)

# Display the map
map_plot_filtered_hotels

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)